In [1]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
from skimage import io
from sklearn import preprocessing
from tqdm.notebook import tqdm, trange
import cv2
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
# # Import spatial omics library
# import athena as ath
# from spatialOmics import SpatialOmics

# # import default graph builder parameters
# from athena.graph_builder.constants import GRAPH_BUILDER_DEFAULT_PARAMS

In [3]:
d_dir = (Path().cwd().parents[0].parents[0]).absolute()
data_dir = d_dir / "09_datasets"

p_dir = (Path().cwd().parents[0]).absolute()

In [4]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)


In [5]:
import graph
import torch
import torch_geometric.utils
import networkx as nx
import lightning.pytorch as pl
import torch.utils.data as data

spatial_omics_folder = (Path().cwd().parents[0]).absolute() / 'data' / 'spatial_omics_graph'
process_path = (Path().cwd().parents[0]).absolute() / 'data' / 'torch_graph_data'
morph_path = (Path().cwd().parents[0]).absolute() / 'data' / 'morph' / 'outlinePCA.pkl'

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create data loader

In [6]:
import graph
import torch

In [7]:
from torch_geometric.loader import DataLoader
seed = torch.Generator().manual_seed(42)

name = 'NIH'

# Crate dataset
dataset = graph.GraphDatasetMorph(process_path / name, morph_path, process_path / name / 'info.csv', 2, y_name='covid')

train_set, val_set, test_set = graph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.nc}')

Dataset: GraphDatasetMorph(442):
Number of graphs: 442
Number of features: 5
Number of classes: 2


In [9]:
print(f'Train set: {len(train_set)}, val set: {len(test_set)}, val set: {len(val_set)}')

Train set: 213, val set: 176, val set: 53


In [10]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    data.label
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 938371], num_nodes=134143, x=[134143, 5], pos=[134143, 2], node_types=[134143], label=[32], covid=[32], train_mask=[134143], test_mask=[134143], y=[32], name=[32], features=[32, 34], features_names=[32], batch=[134143], ptr=[33])



In [11]:
len(dataset[0].features[0])

34

# Train network K Fold

In [12]:
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from torch.utils.data import SubsetRandomSampler

In [14]:
ys = [data.y.item() for data in dataset]

In [15]:
# Print K fold model number of samples and number of positive cases

k_folds = 5
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset, ys)):
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    train_positives = 0
    test_positives = 0
    for step, data in enumerate(train_loader):
        train_positives += np.sum(data.y.numpy())
    for step, data in enumerate(val_loader):
        test_positives += np.sum(data.y.numpy())
        
    print(len(train_subset), train_positives, len(val_subset), test_positives)

309 177 133 76
309 177 133 76
309 177 133 76
309 177 133 76
309 177 133 76


In [16]:
condition = '082423_Covid'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_moprh_{condition}" 
project_name = f'SF_{condition}'

In [17]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'attention', 'attention2']
# models = ['GCN', 'GraphConv', 'GAT', 'GINConv', 'SAGEConv']
pools = ['mean', 'max', 'attention', 'attention2']
# pools = ['attention2']
models = [ 'GAT', 'SAGEConv']
epochs = 100

Global seed set to 42


In [18]:
# Print K fold model number of samples and number of positive cases

# k_folds = 3
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)
fusion_mode = 'concat'
# fusion_mode = 'concat'

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset, ys)):
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    for pool in pools:
        for model in models:
            if model == 'GAT':
                batch_size=64
            else:
                batch_size=128
            # Path to the folder where the pretrained models are saved
            CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}_moprh_{fusion_mode}' / pool
            CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

            # Skip already trained kfold and pool
            checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
            if checkpoint.exists():
                print(checkpoint)
                continue

            # Run training
            run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}_moprh', 
                            group=f'{model}_{pool}_moprh_{fusion_mode}')
            graph.train_graph_classifier_kfold(model, 
                                                 train_subset, 
                                                 val_subset, 
                                                 dataset, 
                                                 CHECKPOINT_PATH, 
                                                 AVAIL_GPUS, 
                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                 num_layers=NUM_LAYERS, 
                                                 epochs=epochs,
                                                 morph=True,
                                                 fusion = fusion_mode,
                                                 batch_size=batch_size,
                                                 graph_pooling=pool)
            run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\pytorch_lightning\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
You are using a CUDA device ('NVIDIA RTX A2000 12GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Training with Morphological features


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_moprh_082423_Covid\GAT_2_16_0_moprh_concat\mean\GraphLevelGAT exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type             | Params
-----------------------------------------------------
0  | feat_embedding | Sequential       | 832   
1  | model          | GAT              | 4.8 K 
2  | head           | Sequential       | 562   
3  | fnn_layer      | Linear           | 272   
4  | selu           | SELU             | 0     
5  | loss_module    | CrossEntropyLoss | 0     
6  | train_acc      | BinaryAccuracy   | 0     
7  | train_auroc    | BinaryAUROC      | 0     
8  | train_f1       | BinaryF1Score    | 0     
9  |

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▃▄▅▅▅▆▆▆▇▇▆▇▇█▇█▇▇█▇▇███▇██▇████████
train_auc,▁▁▂▄▄▅▆▆▆▇▇▇▇▇▇█████▇███████████████████
train_f1,▁▁▂▂▃▃▄▅▄▅▆▆▆▇▆▇▇▇▇█▇▇▇▇▇███▇██▇████████
train_loss_epoch,███▇▆▆▅▄▄▄▃▃▃▂▃▂▂▂▂▁▂▂▂▂▂▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁
train_loss_step,██▇█▆▆▆▅▆▄▃▂▃▂▃▂▃▂▂▃▄▃▂▃▁▁▂▁▂▃▂▂▂▂▂▂▁▂▂▂
trainer/global_step,▁▁▁▁▂▁▂▂▂▂▃▂▃▃▂▄▃▄▃▄▃▅▃▅▄▆▄▆▄▆▄▇▇▅▇▅█▅█▅
val_acc,▁▁▁▁▂▃▄▄▅▃▄▇▇▇▇▇██▇█████▆▆▇▅▅▇▇▆▆▆▇▇▆▄▇▅
val_auc,▁▂▂▂▂▃▄▅▆▆▆▇▇██████████████▇▇▇█▇█▇█████▇
val_f1,▃▃▃▃▄▄▅▅▅▁▃▇▇▇▇▇▇█▇█████▆▆▇▅▅▇█▆▇▇▇█▇▅▇▅


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▂▃▄▄▆▆▆▇▇▇▇▇▇██▇▇█▇▇▇████▇▇▇▇███████
train_auc,▂▁▁▃▄▄▅▅▆▇▇▇████████████████████████████
train_f1,▁▁▂▂▂▃▃▄▅▆▆▆▇▇▇▇▇▇█▇▇▇▇▇▇███▇▇▇▇▇█▇█▇███
train_loss_epoch,████▇▇▆▆▄▃▃▃▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▂▂▂▂▂▁▁▁▁▁▁▁
train_loss_step,█▇▇▇▇▆▅▅▃▂▂▂▃▃▂▂▂▂▁▂▂▂▂▃▃▁▂▂▂▁▁▃▃▂▂▂▂▂▁▂
trainer/global_step,▁▁▁▁▂▁▂▂▂▂▃▂▃▃▂▄▃▄▃▄▃▅▃▅▄▆▄▆▄▆▄▇▇▅▇▅█▅█▅
val_acc,▁▁▁▁▃▄▄▅▆▆▇██▇▇▇▇▇▇▅▇▇▆▅▅█▇▇█▅▆▄▇███▆▇▇▇
val_auc,▁▃▄▄▄▄▅▅▇▇█████████████████████▆████████
val_f1,▁▁▁▁▃▃▄▅▅▆▇▇█▇▇▇▇▆▇▅▇▇▅▅▅▇▇▆█▅▆▃▆██▇▆▆▇▇


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\pytorch_lightning\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_moprh_082423_Covid\SAGEConv_2_16_4_moprh_concat\attention2\GraphLevelSAGEConv exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0

Training with Morphological features


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:281: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchmetrics\utilities\prints.py:42: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▁▂▂▃▃▃▄▄▅▅▆▅▆▆▆▆▆▇▇▇▇▇█████▇██▇█▇▇▇▇██
train_auc,▁▂▁▂▃▃▄▄▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇████████▇███████
train_f1,▁▂▁▂▁▂▂▃▃▄▄▄▆▅▅▅▆▆▆▆▇▇▆▇█▇█▇▇▇██▇█▇▇▇▇██
train_loss_epoch,█████▇▇▇▆▆▅▄▃▄▃▃▃▃▃▃▂▃▃▂▂▁▁▂▂▂▂▁▂▁▂▂▂▂▁▁
train_loss_step,█▇▇▇▇█▅▅▃▄▄▂▃▂▃▃▃▂▂▃▂▂▂▁▂▃▁▂▂▁
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▁▁▂▂▄▄▄▅▅▅▅▅▅▆▆▆▇▆▆▆▇▇▇▇▇█▇▇▇▇▇▆▇▇▆▇█▇▇
val_auc,▁▅▅▄▅▅▅▅▆▆▇▇▇▇▇▇██▇██████████████▇██████
val_f1,▃▃▄▄▁▄▄▄▅▆▆▄▅▅▆▆▆▇▆▆▆▇▇▇▇▇█▇▇▇█▇▇▇▇▇▇█▇▇
